In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

/var/folders/f_/wp8d4zkn51lf_q_66klhkb9r0000gn/T/ipykernel_9242/264531854.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-01-31 09:17:59.665659: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
dataset = pd.read_csv('33847x339_samples.csv', delimiter=',')
dataset.head()

,timestamp,game_time,score,net_worth,team_2_player_0_level,team_2_player_0_kill_count,team_2_player_0_death_count,team_2_player_0_assists_count,team_2_player_0_denies_count,team_2_player_0_lh_count,...,building_27,building_28,building_29,building_30,building_31,building_32,building_33,building_34,building_35,winner
0,1278,961,8,2294,13,6,1,5,10,142,...,0,0,0,0,0,0,0,0,0,radiant
1,1144,849,-1,1261,11,4,1,3,6,84,...,0,0,0,0,0,0,0,0,0,dire
2,1322,1030,4,2241,13,6,1,3,9,159,...,0,0,0,0,0,0,0,0,0,dire
3,1281,1044,-10,-6837,12,3,2,3,4,102,...,0,0,0,0,0,0,0,0,0,dire
4,1178,897,21,13149,10,5,0,3,10,90,...,0,0,0,0,0,0,0,0,0,radiant


In [3]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
dataset['winner'] = label_encoder.fit_transform(dataset['winner'])

In [4]:
X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
ann = tf.keras.models.Sequential()

In [8]:
ann.add(tf.keras.layers.Dense(units=100, activation='relu'))
ann.add(tf.keras.layers.Dense(units=100, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [9]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
952/952 [==============================] - 2s 2ms/step - loss: 0.5372 - accuracy: 0.7077
Epoch 2/100
952/952 [==============================] - 2s 2ms/step - loss: 0.4536 - accuracy: 0.7659
Epoch 3/100
952/952 [==============================] - 4s 4ms/step - loss: 0.3888 - accuracy: 0.8041
Epoch 4/100
952/952 [==============================] - 2s 2ms/step - loss: 0.3353 - accuracy: 0.8340
Epoch 5/100
952/952 [==============================] - 2s 2ms/step - loss: 0.2980 - accuracy: 0.8531
Epoch 6/100
952/952 [==============================] - 2s 2ms/step - loss: 0.2675 - accuracy: 0.8695
Epoch 7/100
952/952 [==============================] - 2s 2ms/step - loss: 0.2428 - accuracy: 0.8812
Epoch 8/100
952/952 [==============================] - 2s 2ms/step - loss: 0.2261 - accuracy: 0.8881
Epoch 9/100
952/952 [==============================] - 2s 2ms/step - loss: 0.2057 - accuracy: 0.8987
Epoch 10/100
952/952 [==============================] - 1s 2ms/step - loss: 0.2022 - accura

In [11]:
y_test

24157    1
17703    1
23474    0
30922    0
4557     1
        ..
28919    1
26345    0
29693    1
19585    1
19297    1
Name: winner, Length: 3385, dtype: int64

In [12]:
y_pred = ann.predict(X_test)
print(y_pred)

106/106 [==============================] - 0s 1ms/step
[[1.0000000e+00]
 [9.9999988e-01]
 [4.0613358e-08]
 ...
 [9.9999124e-01]
 [9.9999911e-01]
 [1.0000000e+00]]


In [13]:
y_pred = y_pred > 0.5
y_pred

array([[ True],
       [ True],
       [False],
       ...,
       [ True],
       [ True],
       [ True]])

In [14]:
from sklearn.metrics import accuracy_score, confusion_matrix
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
print(cm, acc)

[[1353  233]
 [ 204 1595]] 0.8709010339734121


In [15]:
import pymongo

MONGO_CLIENT = pymongo.MongoClient("mongodb://192.168.1.7:27017/")
SDA_DB = MONGO_CLIENT["steam_dota_api"]
MLRS_COL = SDA_DB["multi_real_time_stats"]

In [16]:
# matches = MLRS_COL.find({"_id": ObjectId("65b5e65abb22a1e961d58c20")})
matches = MLRS_COL.find({"match.match_id": "7557978739"})
matches_for_pd = []

for entry in matches:
    try:
        tmp = {}

        tmp["timestamp"] = entry["match"]["timestamp"]
        tmp["game_time"] = entry["match"]["game_time"]

        tmp['score'] = entry["teams"][0]["score"] - entry["teams"][1]["score"]
        tmp['net_worth'] = entry["teams"][0]["net_worth"] - entry["teams"][1]["net_worth"]

        for team in entry["teams"]:
            for p_idx, player in enumerate(team["players"]):
                # tmp["team_{}_player_{}_heroid".format(team["team_number"], p_idx)] = player['heroid']
                tmp["team_{}_player_{}_level".format(team["team_number"], p_idx)] = player['level']
                tmp["team_{}_player_{}_kill_count".format(team["team_number"], p_idx)] = player['kill_count']
                tmp["team_{}_player_{}_death_count".format(team["team_number"], p_idx)] = player['death_count']
                tmp["team_{}_player_{}_assists_count".format(team["team_number"], p_idx)] = player['assists_count']
                tmp["team_{}_player_{}_denies_count".format(team["team_number"], p_idx)] = player['denies_count']
                tmp["team_{}_player_{}_lh_count".format(team["team_number"], p_idx)] = player['lh_count']
                tmp["team_{}_player_{}_gold".format(team["team_number"], p_idx)] = player['gold']
                tmp["team_{}_player_{}_net_worth".format(team["team_number"], p_idx)] = player['net_worth']

                for i_idx, item in enumerate(player['items']):
                    tmp["team_{}_player_{}_item_{}".format(team["team_number"], p_idx, i_idx)] = item

        for gg_idx, graph_gold in enumerate(entry['graph_data']['graph_gold']):
            tmp['graph_gold_{}'.format(gg_idx)] = graph_gold

        for i, building in enumerate(entry["buildings"]):
            tmp["building_{}".format(i)] = int(building["destroyed"])

        matches_for_pd.append(tmp)
    except Exception as e:
        MLRS_COL.delete_one({"_id": entry['_id']})
        print(e, entry['_id'])

In [17]:
df = pd.DataFrame(matches_for_pd)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Columns: 338 entries, timestamp to building_35
dtypes: int64(338)
memory usage: 34.5 KB


In [18]:
X = df.iloc[:, :].values

In [19]:
X_ss = sc.transform(X)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [20]:
y_pred = ann.predict(X_ss)
for y in y_pred:
    if y[0] > 0.5:
        print('radiant', round(y[0], 2))
    else:
        print('dire', round(1- y[0], 2))
    # print(round(1 - y[0], 4), '\t', round(y[0], 4))
    # print(y[0], 1 - y[0])

1/1 [==============================] - 0s 38ms/step
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0


In [21]:
import pickle

with open('ann.pkl','wb') as f:
    pickle.dump(ann, f)

In [22]:
with open('standart_scaler.pkl','wb') as f:
    pickle.dump(sc, f)